In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
from joblib import dump


In [3]:
file = "/Users/liam/PycharmProjects/dataspell/artemis/data/interim/reduced_whois_data.csv"
df = pd.read_csv(file)


/Users/liam/miniconda3/envs/artemis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,23,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,144,145,146,147,148,149,150,151,152,153,155,156) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
label = 'malicious'
labels = df[label].values
data_df = df.drop(columns=label)

In [11]:
cols_to_drop = ['organization', 'email', 'admin_account_name', 'admin_company_name', 'admin_family_name', 'registrant_company_name', 'tech_account_name', 'tech_company_name', 'tech_family_name', 'tech', 'name_servers_11', 'name_servers_12', 'name_servers_13', 'name_servers_14', 'name_servers_15', 'name_servers_16', 'registrant_org', 'referral_url', 'status_7', 'admin_org', 'tech_org', 'status_8', 'curent_registrar', 'registar_created', 'name_server_statuses', 'registrar_site', 'lats_modified', 'updated_date_5', 'registrant_zip', 'status_9', 'status_10', 'status_11', 'status_12', 'emails_7', 'admin_country_code', 'registrar_country_code', 'registrar_organization_id', 'registrar_postal_code', 'registrar_street', 'updated_date_6']
data_df = data_df.drop(columns=cols_to_drop)
data_df = data_df.fillna(-1)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_df, labels, random_state=0)

In [14]:
labelenc

NameError: name 'labelenc' is not defined

In [15]:
labelenc = LabelEncoder().fit(data_df)
X_train[features[0]+"_enc"] = labelenc.transform(X_train[features[0]])
X_train = X_train.drop(columns=features[0])
X_test[features[0]+"_enc"] = labelenc.transform(X_test[features[0]])
X_test = X_test.drop(columns=features[0])


ValueError: y should be a 1d array, got an array of shape (8210, 126) instead.

In [10]:
data_df

,redacted,name_servers_1,name_servers_2,name_servers_3,name_servers_4,registrant_contact_name,registrar,status_1,status_2,status_3,...,updated_date_6,entropy,dns_rec_a_cc,dns_rec_a_org,dns_rec_mx_cc,dns_rec_mx_org,days_between_creations,days_between_updates,days_since_update,days_until_expiration
0,False,T.AU,R.AU,Q.AU,S.AU,CEO,Afilias Australia Pty Ltd,serverDeleteProhibited https://afilias.com.au/...,serverRenewProhibited https://afilias.com.au/g...,serverTransferProhibited https://afilias.com.a...,...,-1,3.337175,au,webcentral noc,au,irt-webcentral-au,-1.0,0.0,21.0,-1.0
1,False,NS33.DOMAINCONTROL.COM,NS34.DOMAINCONTROL.COM,-1,-1,-1,"GoDaddy.com, LLC",clientDeleteProhibited https://icann.org/epp#c...,clientRenewProhibited https://icann.org/epp#cl...,clientTransferProhibited https://icann.org/epp...,...,-1,1.921928,us,"godaddy.com, llc",us,microsoft corporation,0.0,9.0,443.0,470.0
2,False,NS1.DREAMHOST.COM,NS2.DREAMHOST.COM,NS3.DREAMHOST.COM,-1,-1,DREAMHOST,clientTransferProhibited https://icann.org/epp...,clientTransferProhibited https://www.icann.org...,-1,...,-1,2.933955,us,"new dream network, llc",na,na,0.0,0.0,241.0,155.0
3,False,NS1.P23.DYNECT.NET,NS2.P23.DYNECT.NET,NS3.P23.DYNECT.NET,NS4.P23.DYNECT.NET,-1,"GoDaddy.com, LLC",clientDeleteProhibited https://icann.org/epp#c...,clientRenewProhibited https://icann.org/epp#cl...,clientTransferProhibited https://icann.org/epp...,...,-1,3.499228,us,google llc,us,microsoft corporation,0.0,0.0,515.0,216.0
4,False,NS65.WORLDNIC.COM,NS66.WORLDNIC.COM,-1,-1,-1,"Network Solutions, LLC",-1,-1,-1,...,-1,2.947703,na,na,na,na,0.0,0.0,1492.0,2161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8205,True,a.dnspod.com,b.dnspod.com,A.DNSPOD.COM,B.DNSPOD.COM,-1,"NICENIC INTERNATIONAL GROUP CO., LIMITED",clientDeleteProhibited https://icann.org/epp#c...,clientTransferProhibited https://icann.org/epp...,clientHold https://icann.org/epp#clientHold,...,-1,3.000000,na,na,na,na,1.0,17.0,246.0,103.0
8206,True,a.dnspod.com,b.dnspod.com,A.DNSPOD.COM,B.DNSPOD.COM,-1,"NICENIC INTERNATIONAL GROUP CO., LIMITED",clientDeleteProhibited https://icann.org/epp#c...,clientTransferProhibited https://icann.org/epp...,clientHold https://icann.org/epp#clientHold,...,-1,3.000000,na,na,na,na,1.0,17.0,246.0,103.0
8207,True,NS1.DUCKDNS.ORG,NS2.DUCKDNS.ORG,NS3.DUCKDNS.ORG,NS4.DUCKDNS.ORG,-1,GANDI SAS,clientTransferProhibited https://icann.org/epp...,clientTransferProhibited http://www.icann.org/...,-1,...,-1,3.378783,nl,stop the war in ukraine!,nl,stop the war in ukraine!,0.0,0.0,132.0,2562.0
8208,False,NS1.MYHOSTINGPACK.COM,NS2.MYHOSTINGPACK.COM,ns1.myhostingpack.com,ns2.myhostingpack.com,-1,Akky Online Solutions S.A. de C.V.,clientTransferProhibited https://icann.org/epp...,clientTransferProhibited (https://www.icann.or...,-1,...,-1,3.080501,us,"digitalocean, llc",us,"psinet, inc.",0.0,0.0,255.0,108.0


In [8]:
clf = RandomForestClassifier(criterion='entropy', random_state=0).fit(X_train.values, y_train)

In [9]:
clf.score(X_test, y_test)

/Users/liam/miniconda3/envs/artemis/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9778869778869779

In [10]:
model_loc = "../models/"
model_name = "rfc.joblib"
enc_name = "enc.joblib"

In [11]:
dump(clf, model_loc+model_name)

['../models/rfc.joblib']

In [12]:
dump(labelenc, model_loc+enc_name)

['../models/enc.joblib']

In [14]:
clf.feature_importances_

array([0.13983754, 0.30454804, 0.30823104, 0.0337287 , 0.04418349,
       0.16947118])

In [20]:
X_train

,entropy,days_since_creation,year,month,day,dns_rec_mx_cc_enc
5528,3.642490,289.0,2021.0,6.0,12.0,31
3424,2.321928,8691.0,1998.0,6.0,11.0,49
4977,2.845351,-1.0,-1.0,-1.0,-1.0,31
5056,3.327820,147.0,2021.0,11.0,1.0,31
3008,1.584963,-1.0,-1.0,-1.0,-1.0,12
...,...,...,...,...,...,...
4931,3.189898,-1.0,-1.0,-1.0,-1.0,31
3264,3.323074,4507.0,2009.0,11.0,24.0,49
1653,2.921928,9824.0,1995.0,5.0,5.0,49
2607,0.000000,7284.0,2002.0,4.0,18.0,31
